<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [ ]:
!pip install gekko
!pip install ipynb

**Importing libraries needed**

In [4]:
import numpy as np
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAWMILL CLASS**

In [5]:
#SawMill Object
class Mill:
  m = GEKKO(remote=True)

  def __init__(self, Id, Wood, Prod, Dis):
    self.Id = Id
    self.Wood_type =Wood
    self.Prod_type = Prod
    self.Dist = Dis
    #print("Mill: ",self.Id, " Created")

  def Harvest_Close(self, Dem, Available, radio):
    self.Demand = Dem
    Num = 100
    H = m.Array(m.Var,(Num),value = 0)
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = Available[i]
      
    prod0 = m.Intermediate(m.sum([self.Dist[i]*H[i] for i in range(0,len(H))]))
    m.Equation(self.Demand==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=True,debug=True)
    
    return (H)
   

In [6]:
class Inventory:
  def __init__(self, Available, Dist):
    self.Available = Available
    self.Dist = Dist
    self.Mills = []
  
  def Add_Mill(self, m):
    self.Mills.append(m)


**UPLOADING FOREST INVENTORY DATA**

In [7]:
#Upload distances from each hexagon to other
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")



**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [ ]:
#define parameter
Num_Cell= 1000
D = np.zeros(Num_Cell)
Demand = 20000000 
F = Hardwood_Sawlog_Data.SumOfTotal

for i in range(Num_Cell):
  D[i] = i

Hardwood_Sawlog = Inventory(F,D)
a =[]

for k in range(len(Mills_Data)):
  Hardwood_Sawlog.Mills.append(Mill(Mills_Data['Unique_ID'][k],Mills_Data['plant'][k],Mills_Data['s_h'][k],D))

for k in range(5):
  print("Mill ", k)
  Hardwood_Sawlog.Mills[k].Harvest_Close(Demand,F,1)

